In [1]:
import time
import string
import warnings
import numpy as np
import pandas as pd
import requests
import unicodedata

warnings.filterwarnings("ignore")

### Importa dados

In [37]:
municipios = pd.read_csv(r'C:\Users\marci\Downloads/municipios.csv',sep=',')
livrarias = pd.read_csv(r'C:\Users\marci\5.1_Geoloc/lista_livrarias.csv',sep=';')
uf = pd.read_csv(r'C:\Users\marci\Downloads/IBGE_UF_CODIGO.csv',sep=',')

### Funções de suporte

In [54]:
def remover_acentos(texto):
    texto_sem_acento = ''.join(
        char for char in unicodedata.normalize('NFD', texto)
        if unicodedata.category(char) != 'Mn'
    )
    return texto_sem_acento

Visão por municípios

In [55]:
municipios['nome'] = municipios['nome'].apply(lambda x: remover_acentos(x).upper())

municipios2 = municipios.merge(uf, 'left', left_on='codigo_uf', right_on='Código IBGE')
municipios2['chave'] = municipios2.Sigla + '_' + municipios2.nome

cont_livrarias = livrarias.groupby(['uf','municipio'],as_index=False)['razao_social'].count().sort_values(by=['razao_social'],ascending=False).rename(columns={'razao_social':'freq'})
cont_livrarias['chave'] = cont_livrarias.uf + '_' + cont_livrarias.municipio

tbl_merge = cont_livrarias.merge(municipios2, 'left', on='chave')
df = tbl_merge[['uf', 'municipio', 'freq', 'latitude', 'longitude', 'ddd', 'Estado']]
df.dropna(inplace=True)

df['prop'] = (df.freq / df.freq.max()) + 1

Visão por UF

In [171]:
lista_capitais = ['SAO PAULO', 'RIO DE JANEIRO', 'VITORIA', 'BELO HORIZONTE', 'CURITIBA', 'FLORIANOPOLIS', 'PORTO ALEGRE',
           'CUIABA', 'CAMPO GRANDE', 'GOIANIA', 'SALVADOR', 'MACEIO', 'RECIFE', 'FORTALEZA', 'JOAO PESSOA', 'ARACAJU',
            'NATAL','PALMAS', 'TERESINA', 'SAO LUIS', 'PORTO VELHO', 'RIO BRANCO', 'MANAUS', 'BELEM', 'MACAPA', 'BOA VISTA', 'BRASILIA']

cap = municipios[municipios.nome.isin(lista_capitais)]

capitais = cap.merge(uf, 'left', left_on='codigo_uf', right_on='Código IBGE')
capitais2 = capitais[capitais['codigo_ibge'].isin([2800308,1501402,3106200,1400100,5002704,5103403,5208707,4106902,4205407,2304400,2507507,1600303,2704302,1302603,2408102,1721000,4314902,1100205,2611606,1200401,3304557,2927408,2111300,3550308,2211001,3205309, 5300108])]
capitais2.rename(columns={'Sigla':'uf'},inplace=True)

cont_livrarias_uf = livrarias.groupby(['uf'],as_index=False)['razao_social'].count().sort_values(by=['razao_social'],ascending=False).rename(columns={'razao_social':'freq'})

tbl_merge2 = cont_livrarias_uf.merge(capitais2, 'left', on='uf').dropna()
df2 = tbl_merge2[['uf', 'freq', 'latitude', 'longitude', 'ddd', 'Estado']]

df2['prop'] = (df2.freq / df2.freq.max())*9 + 1

In [178]:
tbl = df2
var1 = 'uf'
var2 = 'municipio'

var = var1

Visuzalização do mapa

In [182]:
import folium

mapa = folium.Map(location=[tbl['latitude'].mean(), tbl['longitude'].mean()], zoom_start=4)

folium.TileLayer('cartodbdark_matter').add_to(mapa)
folium.TileLayer('openstreetmap').add_to(mapa)

# Função para criar um mapa com marcadores a partir do DataFrame
def plotar_mapa(df):

    # Adiciona marcadores para cada cidade
    for _, row in df.iterrows():
            folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=row['prop'], fill=True, color='#FFFF00', popup=row[var], tooltip=row[var]).add_to(mapa)

    return mapa

folium.map.LayerControl('topright', collapsed=False).add_to(mapa)

# Exemplo de uso da função:
mapa = plotar_mapa(tbl)
mapa
# mapa.save('mapa.html')  # Salva o mapa em um arquivo HTML